In [1]:
import os

# Set the directory
os.chdir('/Users/ajibolaoluwatobiloba/Desktop/NHL ASSIGNMENT')

# Verify the change
print(os.getcwd())


/Users/ajibolaoluwatobiloba/Desktop/NHL ASSIGNMENT


In [5]:
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel
import matplotlib.pyplot as plt
import os
import tempfile
import nest_asyncio
nest_asyncio.apply()

# Create a temporary directory with correct permissions
output_dir = tempfile.mkdtemp()
print(f"Using temporary directory: {output_dir}")

# Read and prepare the data
df = pd.read_csv('model_inputs.csv')
df['date'] = pd.to_datetime(df['date'])

# Get season from game_id (first 4 digits)
df['season'] = df['game_id'].astype(str).str[:4].astype(int)

# Create a copy for the season data
df_season = df[df['season'] == 2023].copy()

# Create team indices (adding 1 for Stan's 1-based indexing)
home_inds, teams = pd.factorize(df_season['hm_team'], sort=True)
away_inds, _ = pd.factorize(df_season['aw_team'], sort=True)
df_season.loc[:, 'home_inds'] = home_inds + 1
df_season.loc[:, 'away_inds'] = away_inds + 1

# Print basic information
print("Data Loading Complete:")
print(f"Number of games: {len(df_season)}")
print(f"Number of teams: {len(teams)}")
print("\nTeams:", teams.tolist())

# Prepare data for the model
data_sog_xg = {
    'N': len(df_season),
    'n_teams': len(teams),
    'home_team': df_season['home_inds'].values,
    'away_team': df_season['away_inds'].values,
    'home_total_sog': df_season['home_total_shots'].values,
    'away_total_sog': df_season['away_total_shots'].values,
    'home_total_xg': df_season['home_xG'].values,
    'away_total_xg': df_season['away_xG'].values,
    'home_b2b': df_season['hm_is_b2b'].values,
    'away_b2b': df_season['aw_is_b2b'].values
}

# Print some summary statistics
print("\nData Summary:")
print(f"Average home shots: {df_season['home_total_shots'].mean():.2f}")
print(f"Average away shots: {df_season['away_total_shots'].mean():.2f}")
print(f"Average home xG: {df_season['home_xG'].mean():.2f}")
print(f"Average away xG: {df_season['away_xG'].mean():.2f}")

# Initialize the model
print("\nInitializing and fitting the model...")
model_sog_xg = CmdStanModel(
    stan_file='stan-model-sog-xg.stan',
    cpp_options={'STAN_THREADS': True}
)

# Create initialization values
init_values = [
    {
        'sigma_sog': 0.5,
        'sigma_xg': 0.5,
        'alpha_sog': 2.0,
        'alpha_xg': 1.0,
        'p_ha_sog': 0.1,
        'p_ha_xg': 0.1,
        'b2b_effect_sog': -0.1,
        'b2b_effect_xg': -0.1,
        'xg_sigma': 0.5,
        'team_sog_off': list(np.random.normal(0, 0.1, len(teams))),
        'team_sog_def': list(np.random.normal(0, 0.1, len(teams))),
        'team_xg_off': list(np.random.normal(0, 0.1, len(teams))),
        'team_xg_def': list(np.random.normal(0, 0.1, len(teams)))
    } for _ in range(4)  # One for each chain
]

# Fit the model
fit_sog_xg = model_sog_xg.sample(
    data=data_sog_xg,
    chains=4,
    iter_warmup=1000,
    iter_sampling=1000,
    inits=init_values,
    output_dir=output_dir,
    show_progress=True,
    show_console=True
)

# Print model diagnostics
print("\nModel Diagnostics:")
print(fit_sog_xg.diagnose())

# Extract parameters and print summaries
print("\nParameter Summaries:")
print("\nSOG parameters:")
b2b_effect_sog = fit_sog_xg.stan_variable('b2b_effect_sog')
p_ha_sog = fit_sog_xg.stan_variable('p_ha_sog')
print(f"B2B effect on shots: {np.mean(b2b_effect_sog):.3f} ± {np.std(b2b_effect_sog):.3f}")
print(f"Home advantage on shots: {np.mean(p_ha_sog):.3f} ± {np.std(p_ha_sog):.3f}")

print("\nxG parameters:")
b2b_effect_xg = fit_sog_xg.stan_variable('b2b_effect_xg')
p_ha_xg = fit_sog_xg.stan_variable('p_ha_xg')
print(f"B2B effect on xG: {np.mean(b2b_effect_xg):.3f} ± {np.std(b2b_effect_xg):.3f}")
print(f"Home advantage on xG: {np.mean(p_ha_xg):.3f} ± {np.std(p_ha_xg):.3f}")

# Model performance metrics
lambda_sog_home = np.mean(fit_sog_xg.stan_variable('lambda_sog_home'), axis=0)
lambda_sog_away = np.mean(fit_sog_xg.stan_variable('lambda_sog_away'), axis=0)
mu_xg_home = np.mean(fit_sog_xg.stan_variable('mu_xg_home'), axis=0)
mu_xg_away = np.mean(fit_sog_xg.stan_variable('mu_xg_away'), axis=0)

def rmse(actual, predicted):
    return np.sqrt(np.mean((actual - predicted) ** 2))

print("\nModel Performance (RMSE):")
print(f"SOG - Home: {rmse(df_season['home_total_shots'], lambda_sog_home):.3f}")
print(f"SOG - Away: {rmse(df_season['away_total_shots'], lambda_sog_away):.3f}")
print(f"xG - Home: {rmse(df_season['home_xG'], mu_xg_home):.3f}")
print(f"xG - Away: {rmse(df_season['away_xG'], mu_xg_away):.3f}")

# Clean up temporary directory
import shutil
shutil.rmtree(output_dir)

16:35:22 - cmdstanpy - INFO - Chain [1] start processing
16:35:22 - cmdstanpy - INFO - Chain [2] start processing
16:35:22 - cmdstanpy - INFO - Chain [3] start processing
16:35:22 - cmdstanpy - INFO - Chain [4] start processing


Using temporary directory: /var/folders/2g/jqy8zbqj43g1g1t7rhb5xl9h0000gn/T/tmp2futz8m7
Data Loading Complete:
Number of games: 1292
Number of teams: 32

Teams: ['ANA', 'ARI', 'BOS', 'BUF', 'CAR', 'CBJ', 'CGY', 'CHI', 'COL', 'DAL', 'DET', 'EDM', 'FLA', 'LAK', 'MIN', 'MTL', 'NJD', 'NSH', 'NYI', 'NYR', 'OTT', 'PHI', 'PIT', 'SEA', 'SJS', 'STL', 'TBL', 'TOR', 'VAN', 'VGK', 'WPG', 'WSH']

Data Summary:
Average home shots: 7.02
Average away shots: 6.74
Average home xG: 3.19
Average away xG: 2.95

Initializing and fitting the model...
Chain [1] method = sample (Default)
Chain [1] sample
Chain [1] num_samples = 1000 (Default)
Chain [1] num_warmup = 1000 (Default)
Chain [1] save_warmup = false (Default)
Chain [1] thin = 1 (Default)
Chain [1] adapt
Chain [1] engaged = true (Default)
Chain [1] gamma = 0.05 (Default)
Chain [1] delta = 0.8 (Default)
Chain [1] kappa = 0.75 (Default)
Chain [1] t0 = 10 (Default)
Chain [1] init_buffer = 75 (Default)
Chain [1] term_buffer = 50 (Default)
Chain [1] window

16:35:37 - cmdstanpy - INFO - Chain [3] done processing


Chain [3] Iteration: 2000 / 2000 [100%]  (Sampling)
Chain [3] 
Chain [3] Elapsed Time: 8.427 seconds (Warm-up)
Chain [3] 6.481 seconds (Sampling)
Chain [3] 14.908 seconds (Total)
Chain [3] 
Chain [3] 


16:35:37 - cmdstanpy - INFO - Chain [2] done processing
16:35:37 - cmdstanpy - INFO - Chain [1] done processing
16:35:37 - cmdstanpy - INFO - Chain [4] done processing
16:35:37 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lognormal_lpdf: Location parameter is inf, but must be finite! (in 'stan-model-sog-xg.stan', line 83, column 8 to column 63)
	Exception: lognormal_lpdf: Location parameter is inf, but must be finite! (in 'stan-model-sog-xg.stan', line 83, column 8 to column 63)
Exception: lognormal_lpdf: Location parameter is inf, but must be finite! (in 'stan-model-sog-xg.stan', line 83, column 8 to column 63)
	Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stan-model-sog-xg.stan', line 83, column 8 to column 63)


Chain [2] Iteration: 2000 / 2000 [100%]  (Sampling)
Chain [2] 
Chain [2] Elapsed Time: 8.706 seconds (Warm-up)
Chain [2] 6.514 seconds (Sampling)
Chain [2] 15.22 seconds (Total)
Chain [2] 
Chain [2] 
Chain [2] 
Chain [2] 
Chain [2] 
Chain [1] Iteration: 2000 / 2000 [100%]  (Sampling)
Chain [1] 
Chain [1] Elapsed Time: 8.798 seconds (Warm-up)
Chain [1] 6.522 seconds (Sampling)
Chain [1] 15.32 seconds (Total)
Chain [1] 
Chain [1] 
Chain [4] Iteration: 2000 / 2000 [100%]  (Sampling)
Chain [4] 
Chain [4] Elapsed Time: 8.822 seconds (Warm-up)
Chain [4] 6.509 seconds (Sampling)
Chain [4] 15.331 seconds (Total)
Chain [4] 
Chain [4] 

Model Diagnostics:
Processing csv files: /private/var/folders/2g/jqy8zbqj43g1g1t7rhb5xl9h0000gn/T/tmp2futz8m7/stan-model-sog-xg-20241124163522_1.csv, /private/var/folders/2g/jqy8zbqj43g1g1t7rhb5xl9h0000gn/T/tmp2futz8m7/stan-model-sog-xg-20241124163522_2.csv, /private/var/folders/2g/jqy8zbqj43g1g1t7rhb5xl9h0000gn/T/tmp2futz8m7/stan-model-sog-xg-20241124163522_3.cs